In [1]:
!ls data

 age_segment_by_nctid.tsv	    'race_segment_by_nctid (2).tsv'
 nctid_list.tsv			     race_segment_by_nctid.tsv
 nctid_to_pmid_dict.pickle	     raw_data_1960-2022_cui_repleced.zip
 pmed_abstract_cui_replaced_v2.zip


In [2]:
import pickle
import os

In [5]:
vocab_dir = 'data/vocab.txt'
with open(vocab_dir) as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]
print(lines[:10])
print(len(lines))

['[PAD]', '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]']
30522


In [6]:
filename = 'data/cui_voc_all_abtracts_v2.pickle'

with open(filename, 'rb') as handle:
    b = pickle.load(handle)

print(len(b))

11265


In [7]:
blist = list(b)
blist = [x.lower() for x in blist]
blist.sort(key=str.lower)
blist[:10]

['c0000809',
 'c0000814',
 'c0000833',
 'c0000889',
 'c0001126',
 'c0001127',
 'c0001144',
 'c0001145',
 'c0001175',
 'c0001206']

In [8]:
clist = lines + blist 
# clist = blist + lines
print(clist[30520:30542])
print(len(clist))

['##？', '##～', 'c0000809', 'c0000814', 'c0000833', 'c0000889', 'c0001126', 'c0001127', 'c0001144', 'c0001145', 'c0001175', 'c0001206', 'c0001231', 'c0001265', 'c0001306', 'c0001309', 'c0001311', 'c0001314', 'c0001327', 'c0001339', 'c0001342', 'c0001344']
41787


In [10]:
with open('data/vocab_cui_v3.txt', 'w') as f:
    for line in clist:
        f.write(f"{line}\n")

## Process training data

In [29]:
rootdir = 'data/raw_data_1960-2022_cui_repleced_v3'

files_path_list = []
for root, dirs, files in os.walk(rootdir):
    for name in files:
        # print(name)
        if name.endswith((".json")):
            # print(name)
            full_path = os.path.join(root, name)
            files_path_list.append(full_path)

print(len(files_path_list))

549825


In [30]:
import json
from tqdm.auto import tqdm
import random


final_sents_list = []
final_pmid_list = {}
for index, afile in enumerate(tqdm(files_path_list)):
#     print(afile)
    with open(afile, 'r') as f:
        data = json.load(f)
        # print(data)
        data = data[0]
        if 'pmid' in data:
            pmid = data['pmid']
        if 'sentence_list' in data:
            cui_sents = [x['sentence_updated'] for x in data['sentence_list']]
            final_pmid_list[pmid] = cui_sents
            # print(cui_sents)
            for x, y in zip(cui_sents, cui_sents[1:]):
                z = '[CLS] ' + str(x) + ' [SEP] ' + str(y)
                # print(z)
                final_sents_list.append(z)
    if index!=0 and index%10000 == 0:
        print(f'saving {index}')
        final_nonext_list=[]
        key_list = list(final_pmid_list.keys())
        mysize = len(key_list)
        print(f'mysize: {mysize}')
        for i in range(len(final_sents_list)):
            rand1=0
            rand2=0
            while rand1==rand2:
                rand1 = random.choice(range(mysize))
                rand2 = random.choice(range(mysize))
            sent1 = random.choice(final_pmid_list[key_list[rand1]])
            sent2 = random.choice(final_pmid_list[key_list[rand2]])
            z = '[CLS] ' + str(sent1) + ' [SEP] ' + str(sent2)
            final_nonext_list.append(z)
        assert len(final_sents_list) == len(final_nonext_list)
        out_list = [final_sents_list, final_nonext_list]
        out_file_name = 'cui_data_v3_'+str(index)+'.pkl'
        with open(out_file_name, 'wb') as handle:
            pickle.dump(out_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
            print(f'writing file {out_file_name} done')
            handle.close()
        
        final_sents_list = []
        final_pmid_list = {}
        

  0%|          | 0/549825 [00:00<?, ?it/s]

saving 10000
mysize: 10001
writing file cui_data_v3_10000.pkl done
saving 20000
mysize: 10000
writing file cui_data_v3_20000.pkl done
saving 30000
mysize: 10000
writing file cui_data_v3_30000.pkl done
saving 40000
mysize: 10000
writing file cui_data_v3_40000.pkl done
saving 50000
mysize: 10000
writing file cui_data_v3_50000.pkl done
saving 60000
mysize: 10000
writing file cui_data_v3_60000.pkl done
saving 70000
mysize: 10000
writing file cui_data_v3_70000.pkl done
saving 80000
mysize: 10000
writing file cui_data_v3_80000.pkl done
saving 90000
mysize: 10000
writing file cui_data_v3_90000.pkl done
saving 100000
mysize: 10000
writing file cui_data_v3_100000.pkl done
saving 110000
mysize: 10000
writing file cui_data_v3_110000.pkl done
saving 120000
mysize: 10000
writing file cui_data_v3_120000.pkl done
saving 130000
mysize: 10000
writing file cui_data_v3_130000.pkl done
saving 140000
mysize: 10000
writing file cui_data_v3_140000.pkl done
saving 150000
mysize: 10000
writing file cui_data_v3

In [31]:
!zip -r -q cui_data_v3.zip cui_data_v3_* 

In [ ]:
!ls |grep cui_data*

In [28]:
test_file = 'data/raw_data_1960-2022_cui_repleced_v3/7801861.json'  # 7801861  25345354

with open(test_file, 'r') as f:
    data = json.load(f)
#     print(data[0])
    print(data[0]['pmid'])
    print(data[0]['sentence_list'])

7801861
[{'sentence': 'TITLE : Enhanced low-density lipoprotein cholesterol reduction and cost-effectiveness by low-dose colestipol plus lovastatin combination therapy.', 'ent_list': [{'entity_text': 'TITLE', 'label_cui': 'C1705823', 'sem_type': ['T170'], 'start': 0, 'end': 5}, {'entity_text': 'low', 'label_cui': 'C4522223', 'sem_type': ['T170'], 'start': 17, 'end': 20}, {'entity_text': 'lipoprotein cholesterol', 'label_cui': 'C1281910', 'sem_type': ['T059'], 'start': 29, 'end': 52}, {'entity_text': 'reduction', 'label_cui': 'C1293152', 'sem_type': ['T061'], 'start': 53, 'end': 62}, {'entity_text': 'cost', 'label_cui': 'C4050363', 'sem_type': ['T170'], 'start': 67, 'end': 71}, {'entity_text': 'effectiveness', 'label_cui': 'C4553491', 'sem_type': ['T033'], 'start': 72, 'end': 85}, {'entity_text': 'low', 'label_cui': 'C4522223', 'sem_type': ['T170'], 'start': 89, 'end': 92}, {'entity_text': 'colestipol', 'label_cui': 'C0009279', 'sem_type': ['T109', 'T121'], 'start': 98, 'end': 108}, {'e

In [3]:
import pandas as pd

In [19]:
race=pd.read_csv('data/race_segment_by_nctid_before_clean.tsv',sep='\t')
##extract unique segments from age and race
print(race['race'].unique())

['african american' 'asian' 'hispanic' ...
 'oceania (e.g., australian, new zealander, pacific islander)ethnicity (statistics canada)'
 'other (free response)ethnicity (statistics canada)'
 'other north american (e.g., canadian, american, ontarian, quebecois, acadian)ethnicity (statistics canada)']


In [20]:
len(race)

123670

In [21]:
race.dropna(inplace=True)
len(race)

123540

In [24]:
alist = race['race'].unique().tolist()
alist.sort(key=lambda s: len(s))
for i in alist:
    print(i)

no
aa
na
han
yao
yes
ila
hui
man
wei
luo
hoa
iud
hopi
zuni
arab
lozi
zulu
othe
thai
kinh
oral
miao
sikh
none
asia
aian
whte
pill
ring
shot
bozo
peru
igbo
asian
other
white
black
mixed
malay
gipsy
maori
latin
cuban
0-24h
>120h
bemba
chewa
shona
tonga
xhosa
japan
brown
while
hindu
aisan
never
whtie
multi
egypt
qatar
luhya
māori
khmer
lobar
peulh
lulya
patch
china
kazah
tatar
uzbek
thais
hausa
peuhl
gypsy
india
oromo
whites
others
navajo
indian
latino
non-us
korean
native
25-48h
49-72h
73-96h
arabic
lombwe
latina
unkown
syrian
nyanja
blacks
zhuang
lother
europe
latinx
other/
asian.
condom
asian1
taiwan
greeks
slovak
yoruba
russia
uyghur
unknown
missing
african
chinese
asiatic
mestizo
iranian
unkwown
mulatto
negroid
mexican
spanish
97-120h
colored
ndebele
tumbuka
"other"
persian
refused
maghreb
chicano
italian
>1 race
russian
gypsies
condoms
arabian
no data
nothing
turkish
implant
haitian
bambara
malinke
sonrhai
bashkir
songrai
albania
cypriot
madhesi
hispanic
nonwhite
japanese
oriental
eu

In [31]:
matches = ['white', 'black','asian', 'hispanic', 'european', 'africa', 'arab', 'latin', 'native', 'american indian']

def clean_race(text):
    if any(x in text for x in matches):
        return True
    else:
        return False

race_2 = race[race.apply(lambda k: clean_race(k.race), axis=1)].copy()

In [32]:
len(race_2)

90403

In [33]:
alist = race_2['race'].unique().tolist()
alist.sort(key=lambda s: len(s))
for i in alist:
    print(i)

arab
asian
white
black
latin
whites
latino
native
arabic
latina
blacks
latinx
asian.
asian1
african
arabian
hispanic
nonwhite
european
db white
ol white
black-aa
black/aa
latino/a
nh-black
nh-white
aa/black
eurasian
asianole
whiteole
non-white
caucasian
blackrace
not black
whiterace
asianrace
cuacasian
asiangist
blackgist
whitegist
white (w)
non-asian
non-black
not white
causasian
hispanics
asianmain
whitemain
asianmitt
whitemitt
black/ aa
east asian
west asian
race-white
latinorace
white race
caucasians
race,asian
asianwhite
otherwhite
whitewhite
asian east
arabicrace
asianchild
whitechild
latin race
race black
race white
latinarace
asiannurse
blacknurse
whitenurse
hispanic %
arab north
asian (us)
black (us)
white (us)
causcasian
asiantotal
whitetotal
han(asian)
race-asian
race: white
south asian
other asian
asian other
asian: east
east asians
race: black
white/black
africanrace
race: asian
black/other
anglo-asian
asian/other
other-asian
mixed black
other-black
other black
asian (eap)

In [36]:
race = race_2

In [37]:
race.loc[race['nctid'] == 'NCT04365153']  #NCT00000620  NCT00043186 NCT04402970  NCT04365153
# race_2.loc[race_2['nctid'] == 'NCT04365153']  #NCT00000620  NCT00043186 NCT04402970  NCT04365153

,nctid,race,count
121917,NCT04365153,african-american,17.0
121918,NCT04365153,asian,1.0
121919,NCT04365153,caucasian,25.0
121920,NCT04365153,hispanic,2.0


In [39]:
#some preprocessing needed to normalize race
race.loc[ race["race"].str.contains("non-hispanic, white"), "race"] = "white"
race.loc[ race["race"].str.contains("white, non-hispanic"), "race"] = "white"
race.loc[ race["race"].str.contains("nonwhite"), "race"] = "other"
race.loc[ race["race"].str.contains("non-white"), "race"] = "other"
race.loc[ race["race"].str.contains("nonhispanic"), "race"] = "other"
race.loc[ race["race"].str.contains("non-hispanic"), "race"] = "other"
race.loc[ race["race"].str.contains("non-black"), "race"] = "other"
race.loc[ race["race"].str.contains("nonblack"), "race"] = "other"
race.loc[ race["race"].str.contains("black"), "race"] = "black"
race.loc[ race["race"].str.contains("north african"), "race"] = "arab"
race.loc[ race["race"].str.contains("non-arab"), "race"] = "other"
race.loc[ race["race"].str.contains("nonarab"), "race"] = "other"
race.loc[ race["race"].str.contains("arab"), "race"] = "arab"
race.loc[ race["race"].str.contains("non-africa"), "race"] = "other"
race.loc[ race["race"].str.contains("nonafrica"), "race"] = "other"
race.loc[ race["race"].str.contains("africa"), "race"] = "black"
race.loc[ race["race"].str.contains("hispanic"), "race"] = "hispanic"
race.loc[ race["race"].str.contains("non-latin"), "race"] = "other"
race.loc[ race["race"].str.contains("nonlatin"), "race"] = "other"
race.loc[ race["race"].str.contains("latin"), "race"] = "hispanic"
race.loc[ race["race"].str.contains("caucasian"), "race"] = "white"   
race.loc[ race["race"].str.contains("south asian"), "race"] = "south asian"
race.loc[ race["race"].str.contains("non-asian"), "race"] = "other"
race.loc[ race["race"].str.contains("nonasian"), "race"] = "other"
race.loc[ race["race"].str.contains("asian"), "race"] = "asian"
race.loc[ race["race"].str.contains("native"), "race"] = "native"
race.loc[ race["race"].str.contains("american indian"), "race"] = "native"
race.loc[ race["race"].str.contains("white"), "race"] = "white"
race.loc[ race["race"].str.contains("european"), "race"] = "white"

for i in race['race'].unique():
    print(i)

black
asian
hispanic
native
white
other
arab


In [40]:
#create pivot table of NCTID and corresponding patient demographics
race_pivot = pd.pivot_table(race, values='count',index='nctid',columns='race',fill_value=0)
race_pivot.head(5)

race,arab,asian,black,hispanic,native,other,white
nctid,,,,,,,
NCT00000125,0.0,14.0,408.0,59.0,4.0,0.0,1137.0
NCT00000392,0.0,0.0,10.0,25.0,0.0,0.0,177.0
NCT00000620,0.0,0.0,0.0,0.0,0.0,3647.0,6604.0
NCT00001262,0.0,1.0,10.0,0.0,0.0,0.0,44.0
NCT00001304,0.0,0.0,0.0,0.0,0.0,0.0,26.0


In [41]:
race.loc[race['nctid'] == 'NCT04365153']  #NCT00000620  NCT00043186 NCT04402970  NCT04365153

,nctid,race,count
121917,NCT04365153,black,17.0
121918,NCT04365153,asian,1.0
121919,NCT04365153,white,25.0
121920,NCT04365153,hispanic,2.0


In [42]:
nctid_dic = {}
for index, row in race_pivot.iterrows():
    nctid_dic[index] = {'black': row['black'], 'white': row['white']}
nctid_dic

{'NCT00000125': {'black': 408.0, 'white': 1137.0},
 'NCT00000392': {'black': 10.0, 'white': 177.0},
 'NCT00000620': {'black': 0.0, 'white': 6604.0},
 'NCT00001262': {'black': 10.0, 'white': 44.0},
 'NCT00001304': {'black': 0.0, 'white': 26.0},
 'NCT00001305': {'black': 0.0, 'white': 38.0},
 'NCT00001322': {'black': 27.0, 'white': 55.0},
 'NCT00001575': {'black': 15.0, 'white': 66.0},
 'NCT00001586': {'black': 8.0, 'white': 94.0},
 'NCT00001656': {'black': 7.0, 'white': 15.0},
 'NCT00001832': {'black': 1.0, 'white': 109.0},
 'NCT00001849': {'black': 9.0, 'white': 53.0},
 'NCT00001941': {'black': 25.0, 'white': 6.0},
 'NCT00001959': {'black': 8.0, 'white': 8.0},
 'NCT00001984': {'black': 0.0, 'white': 5.0},
 'NCT00002651': {'black': 187.0, 'white': 1026.0},
 'NCT00002850': {'black': 0.0, 'white': 160.0},
 'NCT00003199': {'black': 1.0, 'white': 37.0},
 'NCT00003270': {'black': 2.0, 'white': 18.0},
 'NCT00003590': {'black': 0.0, 'white': 28.0},
 'NCT00003816': {'black': 0.0, 'white': 0.0},

In [43]:
print(len(nctid_dic))

20086


In [44]:
#find pmids associated with NCTIDs
import pickle
# Pickling
pfilename = "data/nctid_to_pmid_dict.pickle"
with open(pfilename,"rb") as file_handle:
    retrieved_data = pickle.load(file_handle)
#     print(retrieved_data)
    print(len(retrieved_data))

count = 0
for k, v in retrieved_data.items():
    count +=1
    print(k, v)
    if count>10:
        break

30151
NCT00833638 []
NCT02424383 []
NCT00919633 []
NCT02314689 []
NCT02111811 ['32108889']
NCT01227824 ['24074642', '23306000']
NCT00071513 ['31228560', '30852711']
NCT02753881 []
NCT01560416 []
NCT03524339 ['33894146']
NCT00612534 ['23271030']


In [45]:
rootdir = 'data/pmed_abstracts_cui_repleced_v2'

cui_dict = {}
for root, dirs, files in os.walk(rootdir):
    for name in files:
        # print(name)
        if name.endswith((".json")):
            # print(name)
            full_path = os.path.join(root, name)
            pmid_name = name.replace('.json', '')
            cui_dict[pmid_name] = full_path

print(len(cui_dict))

12795


In [46]:
for k, v in cui_dict.items():
    print(k, v)
    break

22213153 data/pmed_abstracts_cui_repleced_v2/22213153.json


In [47]:
df = []
seen = set()
for nctid in nctid_dic.keys():
    if nctid in retrieved_data.keys():
        if len(retrieved_data[nctid]) > 0:
            for pmid in retrieved_data[nctid]:
                if pmid in cui_dict.keys():
                    df.append((pmid, nctid_dic[nctid]['black'], nctid_dic[nctid]['white'], nctid, None))
                    seen.add(pmid)
print(len(df))
print(len(seen))
df[0:10]

9369
8615


[('23349433', 408.0, 1137.0, 'NCT00000125', None),
 ('22395889', 408.0, 1137.0, 'NCT00000125', None),
 ('22159015', 408.0, 1137.0, 'NCT00000125', None),
 ('20212196', 408.0, 1137.0, 'NCT00000125', None),
 ('17420364', 408.0, 1137.0, 'NCT00000125', None),
 ('34518266', 0.0, 6604.0, 'NCT00000620', None),
 ('33952397', 0.0, 6604.0, 'NCT00000620', None),
 ('32905623', 0.0, 6604.0, 'NCT00000620', None),
 ('32732305', 0.0, 6604.0, 'NCT00000620', None),
 ('32223381', 0.0, 6604.0, 'NCT00000620', None)]

In [50]:
df = pd.DataFrame(df, columns=['pmid', 'black', 'white', 'nctid', 'ratio'])
df.tail(15)

,pmid,black,white,nctid,ratio
9354,34617959,70.0,102.0,NCT04401293,None
9355,33306283,156.0,496.0,NCT04401579,None
9356,34745093,1.0,24.0,NCT04402970,None
9357,34455583,2.0,19.0,NCT04411628,None
9358,33527249,11.0,81.0,NCT04415489,None
9359,34904132,20.0,21.0,NCT04435184,None
9360,34694880,0.0,7.0,NCT04456634,None
9361,33734539,23.0,131.0,NCT04484207,None
9362,34672949,160.0,584.0,NCT04492475,None
9363,35045900,46.0,256.0,NCT04497415,None


In [51]:
df['ratio'] = df.apply(lambda x: x['black']/max(x['white'], 1),axis=1)

In [52]:
def get_repetitions(black, white):
    ratio = int(black)/max(int(white), 1)
    if ratio == 0:
        return 1
    elif ratio < 0.5:
        return 2
    elif ratio < 1:
        return 4
    else:
        return 8

df['repetition'] = df.apply(lambda x: get_repetitions(x['black'],x['white']),axis=1)

In [58]:
df.loc[df['repetition']==8]

,pmid,black,white,nctid,ratio,repetition
64,23999955,181.0,1.0,NCT00006400,181.000000,8
65,22915643,181.0,1.0,NCT00006400,181.000000,8
66,22294512,181.0,1.0,NCT00006400,181.000000,8
67,22190441,181.0,1.0,NCT00006400,181.000000,8
68,22012708,181.0,1.0,NCT00006400,181.000000,8
...,...,...,...,...,...,...
9256,34851411,61.0,32.0,NCT03818815,1.906250,8
9281,31444872,19.0,10.0,NCT03898349,1.900000,8
9312,34793441,40.0,7.0,NCT04150250,5.714286,8
9349,33347320,8767.0,0.0,NCT04371419,8767.000000,8


In [59]:
df.to_csv('data/ncit_pmid_df_simple.tsv', sep='\t', index=False)

In [71]:
print(df['pmid'].unique())
print(df['pmid'].nunique())

['23349433' '22395889' '22159015' ... '33308429' '33797063' '34887544']
8615


In [61]:
pmid_repeat_dict = dict(zip(df['pmid'], df['repetition']))
for k, v in pmid_repeat_dict.items():
    print(k, v)
    break

23349433 2


In [62]:
'23349433' in pmid_repeat_dict

True

In [208]:
mydf=pd.read_csv('data/ncit_pmid_df_simple.tsv', sep='\t')

In [209]:
mydf.head()

,pmid,black,white,nctid,ratio,repetition
0,23349433,408.0,1137.0,NCT00000125,0.358839,2
1,22395889,408.0,1137.0,NCT00000125,0.358839,2
2,22159015,408.0,1137.0,NCT00000125,0.358839,2
3,20212196,408.0,1137.0,NCT00000125,0.358839,2
4,17420364,408.0,1137.0,NCT00000125,0.358839,2


In [219]:
mydf.pmid.nunique()

8615

In [218]:
mydf.nctid.nunique()

5421

In [220]:
print(mydf.loc[mydf['ratio']<0.1]['nctid'].nunique())

print(mydf.loc[(mydf['ratio']>0.1) & (mydf['ratio']<0.2)]['nctid'].nunique())

print(mydf.loc[(mydf['ratio']>0.2) & (mydf['ratio']<0.4)]['nctid'].nunique())

print(mydf.loc[(mydf['ratio']>0.4) & (mydf['ratio']<0.8)]['nctid'].nunique())

print(mydf.loc[mydf['ratio']>0.8]['nctid'].nunique())

2912
711
695
412
640


In [223]:
print(2912/5421)
print(711/5421)
print(695/5421)
print(412/5421)
print(640/5421)

0.5371702637889688
0.13115661317100166
0.1282051282051282
0.07600073787124147
0.11805939863493821


In [221]:
print(mydf.loc[mydf['ratio']<0.1]['pmid'].nunique())

print(mydf.loc[(mydf['ratio']>0.1) & (mydf['ratio']<0.2)]['pmid'].nunique())

print(mydf.loc[(mydf['ratio']>0.2) & (mydf['ratio']<0.4)]['pmid'].nunique())

print(mydf.loc[(mydf['ratio']>0.4) & (mydf['ratio']<0.8)]['pmid'].nunique())

print(mydf.loc[mydf['ratio']>0.8]['pmid'].nunique())

4951
1109
1055
598
1014


In [224]:
print(4951/8615)
print(1109/8615)
print(1055/8615)
print(598/8615)
print(1014/8615)


0.5746952988972722
0.12872896111433546
0.122460824143935
0.069413813116657
0.11770168311085316


In [82]:
len(df)

9369

In [83]:
import json
from tqdm.auto import tqdm
import random
 
    
def get_title_and_abstract(data):
    pmid = data['pmid']
    sentences = data['sentence_list']
    title = ''
    abstract_sents = ''
    for x in sentences:
        asent = x['sentence']
        if asent.startswith('TITLE :'):
            title = asent.replace('TITLE : ', '')
        else:
            abstract_sents += asent
    return title, abstract_sents
    
# rootdir = 'data/raw_data_1960-2022_cui_repleced_v2'
rootdir = 'data/pmed_abstracts_cui_repleced_v2'

# name = '22159015'

    
def read_pmid_from_file(row):
    title = ''
    abstract_sents = ''
    name = str(row['pmid'])
    afile = os.path.join(rootdir, name +'.json')
    with open(afile, 'r') as f:
        data = json.load(f)
        if len(data)>0:
            data = data[0]
            title, abstract_sents = get_title_and_abstract(data)
    return title, abstract_sents
    

# read_pmid_from_file(df.iloc[2,:])
    
df[["title","abstract"]] = df.apply(lambda x: read_pmid_from_file(x), axis=1, result_type="expand")





In [90]:
df.head()

,pmid,black,white,nctid,ratio,repetition,title,abstract
0,23349433,408.0,1137.0,NCT00000125,0.358839,2,Series length used during trend analysis affec...,OBJECTIVE : Trend analysis techniques to detec...
1,22395889,408.0,1137.0,NCT00000125,0.358839,2,Effect of treatment on the rate of visual fiel...,OBJECTIVE : The goal in this study was to comp...
2,22159015,408.0,1137.0,NCT00000125,0.358839,2,The rate of visual field change in the ocular ...,OBJECTIVE : To assess the rate of change of vi...
3,20212196,408.0,1137.0,NCT00000125,0.358839,2,Delaying treatment of ocular hypertension: the...,OBJECTIVE : To compare the safety and efficacy...
4,17420364,408.0,1137.0,NCT00000125,0.358839,2,Comparison of initial intraocular pressure res...,OBJECTIVE : To compare the intraocular pressur...


In [93]:
df_simple = df[ ['pmid','black','white','nctid','title','abstract']].copy()
print(len(df_simple))
df_simple.tail(10)

9369


,pmid,black,white,nctid,title,abstract
9359,34904132,20.0,21.0,NCT04435184,"Effect of Crizanlizumab, a P-Selectin Inhibito...",COVID-19 is characterized by vascular inflamma...
9360,34694880,0.0,7.0,NCT04456634,"Safety, Tolerability, Pharmacokinetics, and Ph...","Despite repeated malaria infection, individual..."
9361,33734539,23.0,131.0,NCT04484207,Increasing treatment-seeking intentions of US ...,BACKGROUND : Veterans have higher rates of anx...
9362,34672949,160.0,584.0,NCT04492475,Efficacy of interferon beta-1a plus remdesivir...,BACKGROUND : Functional impairment of interfer...
9363,35045900,46.0,256.0,NCT04497415,Video intervention to increase treatment-seeki...,BACKGROUND : Many healthcare workers do not se...
9364,34259846,7879.0,9810.0,NCT04502056,Effect of Physician-Delivered COVID-19 Public ...,Importance : Social distancing is critical to ...
9365,34551869,0.0,0.0,NCT04525079,"Safety, Virologic Efficacy, and Pharmacokineti...",PURPOSE : Neutralizing antibodies can reduce S...
9366,33308429,1.0,243.0,NCT04544787,Safety and immunogenicity of two novel type 2 ...,BACKGROUND : Two novel type 2 oral poliovirus ...
9367,33797063,10.0,38.0,NCT04570670,Pharmacokinetics and Bioavailability of Monome...,BACKGROUND : Tecfidera® (dimethyl fumarate [DM...
9368,34887544,257.0,1632.0,NCT04634903,A randomized trial of online single-session in...,The COVID-19 pandemic has potentially increase...


In [94]:
df_simple.to_csv('data/ncit_pmid_df_title_abstract_0810.tsv', sep='\t', index=False)

In [99]:
df_simple.to_csv('data/abstracts_and_population_0810.tsv', sep='\t', index=False)

## generate data

In [97]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.9 MB/s eta 0:00:00a 0:00:01


In [98]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import sent_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords

from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from nltk.corpus import wordnet
import re
import string

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    if treebank_tag.startswith('V'):
        return wordnet.VERB
    if treebank_tag.startswith('N'):
        return wordnet.NOUN
    if treebank_tag.startswith('R'):
        return wordnet.ADV
    return wordnet.NOUN
        # return ''

def is_english_word(word_to_check):
    return len(wordnet.synsets(word_to_check)) > 0


def represents_number(word_to_check):
    possible_numbers = re.findall(r'[\d\.]+', word_to_check)
    if len(possible_numbers) > 0 and len(possible_numbers[0]) == len(word_to_check):
        return True
    syns = wordnet.synsets(word_to_check)
    for s in syns:
        if s.definition().startswith("the cardinal number"):
            return True
    if "-" in word_to_check:
        word = word_to_check.split("-")[0]
        syns = wordnet.synsets(word)
        for s in syns:
            if s.definition().startswith("the cardinal number"):
                return True
    return False


def stem_carefully(word):
    stemmer = SnowballStemmer('english')
    stemmed = stemmer.stem(word)
    if is_english_word(stemmed):
        return stemmed
    else:
        return word


def clean(text):
    """tokenize, remove stop words and numbers, and lemmatize."""
    stop_words = stopwords.words('english')

    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    pos = pos_tag(tokens)
    lemm = [lemmatizer.lemmatize(word, pos=get_wordnet_pos(p)).lower() for word, p in pos]
    without_stop_words = [word for word in lemm
                          if word not in stop_words
                          and word not in string.punctuation
                          and not represents_number(word)
                          and is_english_word(word)]
    return without_stop_words


def decide_on_word(word):
    w = word.lower().strip(string.punctuation)
    if represents_number(w):
        return "<NUMBER>"
    return w


def break_and_lemmatize(text):
    """tokenize, remove stop words and numbers, and lemmatize."""
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    # some words have a slash that is not caught by word_tokenize
    tokens1 = []
    for token in tokens:
        s = [t for t in token.strip(string.punctuation).split("/") if t != ""]
        tokens1.extend(s)
    pos = pos_tag(tokens1)
    lemm = [lemmatizer.lemmatize(word, pos=get_wordnet_pos(p)).lower() for word, p in pos]
    without = [decide_on_word(word) for word in lemm if word not in string.punctuation]
    return without

def flatten_list_of_list_of_texts(lst):
    return [item.strip() for sublist in lst for item in sublist]

def sentence_break(text):
    return flatten_list_of_list_of_texts([sent_tokenize(x) for x in text.split(";")])


def split_abstract_to_sentences(abstract):
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    parts = abstract.split(';')
    sentences = []
    for part in parts:
        sentences.extend([s.strip('[]()') for s in sent_tokenizer.tokenize(part)])
    return sentences

In [104]:
import re
import string
import time
import pandas as pd
tqdm.pandas()

In [102]:
def should_keep_sentence(sentence):
    blacklist = ['http', 'https', 'url', 'www', 'clinicaltrials.gov', 'copyright', 'funded by', 'published by', 'subsidiary', '©', 'all rights reserved']
    s = sentence.lower()
    for w in blacklist:
        if w in s:
            return False
    # re, find NCTs
    if len(re.findall('nct[0-9]+', s)) > 0:
        return False
    if len(s) < 10:
        return False
    return True


def clean_abstracts(df):
    # filter sentences
    df['sentences'] = df['title_and_abstract'].apply(split_abstract_to_sentences)
    d = {'total': 0, 'remaining': 0}

    def pick_sentences(sentences):
        new_sents = [sent for sent in sentences if should_keep_sentence(sent)]
        d['total'] += len(sentences)
        d['remaining'] += len(new_sents)
        return new_sents

    def join_to_abstract(sentences):
        return ' '.join(sentences)

    df['sentences'] = df['sentences'].apply(pick_sentences)
    df['title_and_abstract'] = df['sentences'].apply(join_to_abstract)
    print(f"kept {d['remaining']}/{d['total']} sentences without blacklisted words")
    return df


def read_abstracts_file(fname):
    df = pd.read_csv(fname, index_col=0, sep='\t')
    df['title'] = df['title'].fillna('')
    df['title'] = df['title'].apply(lambda x: x.strip('[]'))
    df['title_and_abstract'] = df['title'] + ' ' + df['abstract']
    df = df.dropna(axis=0, subset=['title_and_abstract'])
    df = clean_abstracts(df)
    return df


def tokenize_title_and_abstract(input_file, output_file="abstracts_and_population_output.tsv"):

    def preprocess_row(row):
        text = " ".join([row['title'], row['abstract']])
        text = text.replace(";", " ")
        return break_and_lemmatize(text)

    df = read_abstracts_file(input_file)
    print("tokenizing")
    df['tokenized'] = df.progress_apply(preprocess_row, axis=1)
    df.to_csv(output_file,sep='\t')

In [110]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/hl2659/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/hl2659/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/hl2659/nltk_data...


True

In [111]:
input_file = 'data/abstracts_and_population_0810.tsv'
output_file="data/abstracts_and_population_output_0810.tsv"

tokenize_title_and_abstract(input_file, output_file)

kept 113174/129499 sentences without blacklisted words
tokenizing


100%|██████████| 9369/9369 [04:18<00:00, 36.25it/s]


In [112]:
output_file="data/abstracts_and_population_output_0810.tsv"
new_df = pd.read_csv(output_file, sep='\t')
new_df.head()

,pmid,black,white,nctid,title,abstract,title_and_abstract,sentences,tokenized
0,23349433,408.0,1137.0,NCT00000125,Series length used during trend analysis affec...,OBJECTIVE : Trend analysis techniques to detec...,Series length used during trend analysis affec...,['Series length used during trend analysis aff...,"['series', 'length', 'use', 'during', 'trend',..."
1,22395889,408.0,1137.0,NCT00000125,Effect of treatment on the rate of visual fiel...,OBJECTIVE : The goal in this study was to comp...,Effect of treatment on the rate of visual fiel...,['Effect of treatment on the rate of visual fi...,"['effect', 'of', 'treatment', 'on', 'the', 'ra..."
2,22159015,408.0,1137.0,NCT00000125,The rate of visual field change in the ocular ...,OBJECTIVE : To assess the rate of change of vi...,The rate of visual field change in the ocular ...,['The rate of visual field change in the ocula...,"['the', 'rate', 'of', 'visual', 'field', 'chan..."
3,20212196,408.0,1137.0,NCT00000125,Delaying treatment of ocular hypertension: the...,OBJECTIVE : To compare the safety and efficacy...,Delaying treatment of ocular hypertension: the...,['Delaying treatment of ocular hypertension: t...,"['delaying', 'treatment', 'of', 'ocular', 'hyp..."
4,17420364,408.0,1137.0,NCT00000125,Comparison of initial intraocular pressure res...,OBJECTIVE : To compare the intraocular pressur...,Comparison of initial intraocular pressure res...,['Comparison of initial intraocular pressure r...,"['comparison', 'of', 'initial', 'intraocular',..."


In [119]:
new_df.head()

,pmid,black,white,nctid,title,abstract,title_and_abstract,sentences,tokenized
0,23349433,408.0,1137.0,NCT00000125,Series length used during trend analysis affec...,OBJECTIVE : Trend analysis techniques to detec...,Series length used during trend analysis affec...,['Series length used during trend analysis aff...,"['series', 'length', 'use', 'during', 'trend',..."
1,22395889,408.0,1137.0,NCT00000125,Effect of treatment on the rate of visual fiel...,OBJECTIVE : The goal in this study was to comp...,Effect of treatment on the rate of visual fiel...,['Effect of treatment on the rate of visual fi...,"['effect', 'of', 'treatment', 'on', 'the', 'ra..."
2,22159015,408.0,1137.0,NCT00000125,The rate of visual field change in the ocular ...,OBJECTIVE : To assess the rate of change of vi...,The rate of visual field change in the ocular ...,['The rate of visual field change in the ocula...,"['the', 'rate', 'of', 'visual', 'field', 'chan..."
3,20212196,408.0,1137.0,NCT00000125,Delaying treatment of ocular hypertension: the...,OBJECTIVE : To compare the safety and efficacy...,Delaying treatment of ocular hypertension: the...,['Delaying treatment of ocular hypertension: t...,"['delaying', 'treatment', 'of', 'ocular', 'hyp..."
4,17420364,408.0,1137.0,NCT00000125,Comparison of initial intraocular pressure res...,OBJECTIVE : To compare the intraocular pressur...,Comparison of initial intraocular pressure res...,['Comparison of initial intraocular pressure r...,"['comparison', 'of', 'initial', 'intraocular',..."


## Training embeddings

In [118]:
!pip install gensim
!pip install scipy
!pip install scikit-learn


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [127]:
from ast import literal_eval
import numpy as np

In [120]:
READ_MODELS = False
SAVE_MODELS = True

ONLY_CONCEPTS = False
MODE = "plain_text"  # 'plain_text', 'cui2vec_style' or 'only_concepts'
MIN_COUNT = 0
MIN_COUNT_BEFORE_REP = 0 # 40
VECTOR_SIZE = 40  # 50, 70, 100, 300
TRAINING_ITERATIONS = 15  # 35
WINDOW = 10 # 20
TOPN = 10

In [121]:
white_model_file_template = "embedding_models/white{}.model"
black_model_file_template = "embedding_models/black{}.model"
neutral_model_file_template = "embedding_models/neutral{}.model"

In [122]:
mode_to_filename = {'plain_text': 'data/abstracts_and_population_output_0810.tsv',
                    'cui2vec_style': 'abstracts_and_population_tokenized_for_cui2vec.csv',
                    'only_concepts': 'abstracts_and_population_tokenized_cui.csv',}

In [123]:
def read_tokenized_df(mode='plain_text', print_stats=False):
    # mode: 'only_concepts'/ 'plain_text'/ 'cui2vec_style'
    filename = mode_to_filename[mode]
    print("reading {}".format(filename))
    df = pd.read_csv(filename, index_col=0, sep='\t')
    if mode == 'cui2vec_style':
        df = df.dropna(axis=0, subset=['tokenized_sents'])
        df['tokenized_sents'] = df['tokenized_sents'].apply(literal_eval)
        df['tokenized'] = df['tokenized_sents'].apply(lambda sents: " ".join(sents))
        df['tokenized'] = df['tokenized'].apply(lambda x: x.split())
    else:
        df = df.dropna(axis=0, subset=['tokenized'])
        df['tokenized'] = df['tokenized'].progress_apply(literal_eval)
    if mode == 'only_concepts' and FILTER_BY_SEMTYPES:
        print("Filtering by semtypes to keep only: {}".format(SEMANTIC_TYPES))
        # cui_mapper = CUIMapper("cui_table.tsv")

        def filter_by_semtypes(row):
            return [cui for cui in row if cui_mapper.cui_matches_semtypes(cui, SEMANTIC_TYPES)]

        df['tokenized'] = df['tokenized'].progress_apply(filter_by_semtypes)
    if print_stats:
        print("Corpus stats:")
        print("# abstracts: {}".format(len(df)))
        list_of_lists = df['tokenized'].values.tolist()
        all_words = [item for sublist in list_of_lists for item in sublist]
        print("# words: {}".format(len(all_words)))
        unique_words = set(all_words)
        print("# unique words: {}".format(len(unique_words)))
        cuis = [w for w in all_words if w.startswith('C') and len(w) == 8 and w[1:].isdigit()]
        print("# cuis: {}".format(len(cuis)))
        print("# unique cuis: {}".format(len(set(cuis))))
        print("calculating length of abstracts")

    lens = df['tokenized'].progress_apply(len)
    set_lens = df['tokenized'].progress_apply(lambda x: len(set(x)))
    print("average length of abstract: {}, max length: {}, min length: {}".format(np.mean(lens), max(lens), min(lens)))
    print("without repetitions in one abstract: average length: {} max length: {}, min length: {}".format(
        np.mean(set_lens), max(set_lens), min(set_lens)))

    return df

In [131]:
from collections import defaultdict

def word_count(list_of_word_lists):
    word_to_count = defaultdict(int)
    for lst in list_of_word_lists:
        for w in lst:
            word_to_count[w] += 1
    return word_to_count

In [132]:
df = read_tokenized_df(MODE)
word_counter = word_count(df['tokenized'])
num_words = len([w for w in word_counter.keys() if (word_counter[w] > MIN_COUNT_BEFORE_REP) and w.startswith('C') and len(w) == 8])
print("mode: {}, unique words: {}, unique CUIs: {} ".format(MODE, len(word_counter), num_words))
df['all_participants'] = df['black']+df['white']

reading data/abstracts_and_population_output_0810.tsv


100%|██████████| 9369/9369 [00:00<00:00, 27054.81it/s]


average length of abstract: 326.2285195858683, max length: 1167, min length: 15
without repetitions in one abstract: average length: 141.6061479346782 max length: 356, min length: 15
mode: plain_text, unique words: 72646, unique CUIs: 0 


In [178]:
import numpy as np
from gensim.models import Word2Vec
from scipy.spatial.distance import cosine as cos_dist


class EmbedModelWrapper(object):

    def __init__(self, min_count, vector_size, iterations, window):
        self.w2v_model = None
        self.min_count = min_count
        self.vector_size = vector_size
        self.iterations = iterations
        self.window = window

    def train(self, tokenized_abstracts):
        # @param tokenized_abstracts - a list of lists of strings
        # Based on this: https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92
        print("training model")
        self.w2v_model = Word2Vec(
            tokenized_abstracts, min_count=self.min_count, vector_size=self.vector_size,
            workers=3, window=self.window, sg=0, epochs=self.iterations)

    def save(self, filename):
        # save both models
        self.w2v_model.save(filename + "_w2v")


    def load(self, filename):
        # load both models
        self.w2v_model = Word2Vec.load(filename + "_w2v")
        
    def similarity(self, w1, w2):
        return self.w2v_model.wv.similarity(w1, w2)

    def get_vocab(self):
        return self.w2v_model.wv.key_to_index

    def get_vocab_list(self):
        return list(self.w2v_model.wv.key_to_index.keys())

    def get_count(self, w):
        if w in self.w2v_model.wv.key_to_index:
            return self.w2v_model.wv.key_to_index[w].count
        return 0

    def most_similar(self, word, topn=5):
        top = []
        # Or use w2v
        if word in self.w2v_model.wv.key_to_index:
            top = self.w2v_model.wv.most_similar(positive=word, topn=topn)
        return top

    def get_word_vectors_as_matrix_rows(self, index_to_word):
        """index_to_word - a list of words in the required order"""
        vectors = []
        for w in index_to_word:
            vectors.append(self.w2v_model.wv[w])
        return np.array(vectors)

    def export_embeddings(self, output_file):
        """model is an instance of EmbedModelWrapper"""
        vocab = self.get_vocab_list()
        matrix = self.get_word_vectors_as_matrix_rows(vocab)
        out = open(output_file, "w")
        for i in range(len(vocab)):
            vector_as_str = ",".join([str(x) for x in matrix[i]])
            out.write("{}\t{}\n".format(vocab[i], vector_as_str))


            
            

# class ModelMerger(EmbedModelWrapper):
#     def __init__(self, models):
#         self.index_to_word, self.word_to_index, self.counter = self.merge_vocabs(models)
#         self.sim = self.calc_similarity_matrix(models)

#     def merge_vocabs(self, models):
#         word_set = set(models[0].get_vocab())
#         for model in models[1:]:
#             before = len(word_set)
#             word_set = word_set.union(set(model.get_vocab()))
#             after = len(word_set)
#             if after > before:
#                 print("Warning: found misatching vocabs in merger.")
#         index_to_word = list(word_set)
#         word_to_index = {index_to_word[index]: index for index in range(len(index_to_word))}
#         counter = {index_to_word[index]: models[0].get_count(index_to_word[index]) for index in range(len(index_to_word))}
#         return index_to_word, word_to_index, counter

#     def calc_similarity_matrix(self, models):
#         matrices = []
#         for model in models:
#             m1 = model.get_word_vectors_as_matrix_rows(self.index_to_word)
#             m2 = m1 / np.linalg.norm(m1, axis=1)[:, np.newaxis]
#             matrices.append(np.dot(m2, m2.transpose()))
#         return np.mean(np.array(matrices), axis=0)

#     def most_similar(self, w, topn=5):
#         if w in self.word_to_index:
#             sim_to_w = self.sim[self.word_to_index[w]]
#             word_ids = np.argsort(-sim_to_w)
#             # the first will likely be the original word
#             neighbors = [(self.index_to_word[x], sim_to_w[x]) for x in word_ids[:topn+1]]
#             if neighbors[0][0] == w:
#                 neighbors = neighbors[1:]
#             return neighbors
#         return []

#     def get_vocab(self):
#         return self.word_to_index

#     def get_vocab_list(self):
#         return self.index_to_word

#     def similarity(self, w1, w2):
#         return self.sim[self.word_to_index[w1], self.word_to_index[w2]]

#     def get_count(self, w):
#         return self.counter.get(w, 0)

In [194]:
from gensim.models import FastText
class FastTextEmbedModelWrapper(object):

    def __init__(self, min_count, vector_size, iterations, window):
        self.w2v_model = None
        self.min_count = min_count
        self.vector_size = vector_size
        self.iterations = iterations
        self.window = window

    def train(self, tokenized_abstracts):
        # @param tokenized_abstracts - a list of lists of strings
        # Based on this: https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92
        print("training model")
        self.w2v_model = FastText(
            tokenized_abstracts, min_count=self.min_count, vector_size=self.vector_size,
            workers=3, window=self.window, sg=0, epochs=self.iterations)

    def save(self, filename):
        # save both models
        self.w2v_model.save(filename + "_ft")


    def load(self, filename):
        # load both models
        self.w2v_model = FastText.load(filename + "_ft")
        
    def similarity(self, w1, w2):
        return self.w2v_model.wv.similarity(w1, w2)

    def get_vocab(self):
        return self.w2v_model.wv.key_to_index

    def get_vocab_list(self):
        return list(self.w2v_model.wv.key_to_index.keys())

    def get_count(self, w):
        if w in self.w2v_model.wv.key_to_index:
            return self.w2v_model.wv.key_to_index[w].count
        return 0

    def most_similar(self, word, topn=5):
        top = []
        # Or use w2v
        if word in self.w2v_model.wv.key_to_index:
            top = self.w2v_model.wv.most_similar(positive=word, topn=topn)
        return top

    def get_word_vectors_as_matrix_rows(self, index_to_word):
        """index_to_word - a list of words in the required order"""
        vectors = []
        for w in index_to_word:
            vectors.append(self.w2v_model.wv[w])
        return np.array(vectors)

    def export_embeddings(self, output_file):
        """model is an instance of EmbedModelWrapper"""
        vocab = self.get_vocab_list()
        matrix = self.get_word_vectors_as_matrix_rows(vocab)
        out = open(output_file, "w")
        for i in range(len(vocab)):
            vector_as_str = ",".join([str(x) for x in matrix[i]])
            out.write("{}\t{}\n".format(vocab[i], vector_as_str))

In [180]:
iterations = ['_plain_text_w2v_vec40_win10_ti15']
neutral_models, white_models, black_models = [], [], []
for iteration in iterations:
    white_model_file = white_model_file_template.format(iteration)
    black_model_file = black_model_file_template.format(iteration)
    neutral_model_file = neutral_model_file_template.format(iteration)

    wm = EmbedModelWrapper(min_count=MIN_COUNT,
                           vector_size=VECTOR_SIZE, iterations=TRAINING_ITERATIONS,
                           window=WINDOW)
    bm = EmbedModelWrapper(min_count=MIN_COUNT,
                           vector_size=VECTOR_SIZE, iterations=TRAINING_ITERATIONS,
                           window=WINDOW)
    neutral = EmbedModelWrapper(min_count=MIN_COUNT,
                                vector_size=VECTOR_SIZE, iterations=TRAINING_ITERATIONS,
                                window=WINDOW)

In [195]:
iterations = ['_plain_text_FT_vec40_win10_ti15']
neutral_models_ft, white_models_ft, black_models_ft = [], [], []
for iteration in iterations:
    white_model_file_ft = white_model_file_template.format(iteration)
    black_model_file_ft = black_model_file_template.format(iteration)
    neutral_model_file_ft = neutral_model_file_template.format(iteration)

    wm_ft = FastTextEmbedModelWrapper(min_count=MIN_COUNT,
                           vector_size=VECTOR_SIZE, iterations=TRAINING_ITERATIONS,
                           window=WINDOW)
    bm_ft = FastTextEmbedModelWrapper(min_count=MIN_COUNT,
                           vector_size=VECTOR_SIZE, iterations=TRAINING_ITERATIONS,
                           window=WINDOW)
    neutral_ft = FastTextEmbedModelWrapper(min_count=MIN_COUNT,
                                vector_size=VECTOR_SIZE, iterations=TRAINING_ITERATIONS,
                                window=WINDOW)

In [170]:

def participants_to_repetitions(participants):
    if participants == 0:
        repetitions = 0
    elif participants < 10:
        repetitions = 1
    elif participants < 100:
        repetitions = 10
    else:
        repetitions = 20
    return repetitions


def repeat_by_participants(df, weight_col, word_counter, threshold,
                           policy=participants_to_repetitions):
    tokenized = []
    reps = defaultdict(int)
    total_len = 0
    for i, r in df.iterrows():
        word_list = r['tokenized']
        remaining = [word for word in word_list if word_counter[word] >= threshold]
        if len(remaining) < 2:
            reps['empty'] += 1
            continue
        total_len += len(remaining)
        repetitions = policy(r[weight_col])
        #DEBUG
        reps[repetitions] += 1
        for _ in range(repetitions):
            tokenized.append(remaining)
    # DEBUG
    print("repetitions histogram: {}".format(reps))
    remaining_abstracts = len(df)-reps['empty']
    print("remaining abstracts after removing rare words and then empty abstracts: {}, avg length: {}".format(
          remaining_abstracts,
          total_len/remaining_abstracts))
    random.shuffle(tokenized)
    return tokenized

In [156]:
model_path = 'embedding_models'
os.makedirs(model_path, exist_ok=True)
!ls


'BERT with new voc-Copy1.ipynb'   hao_test_v1.ipynb
 data				 'Length Hospital Stay Prediction.ipynb'
 embedding_models		  output_sensitive
 exBERT				  pytorch_model.bin


In [182]:
if READ_MODELS:
    wm.load(white_model_file)
    bm.load(black_model_file)
    neutral.load(neutral_model_file)
else:
    print("Training white")
    wm.train(repeat_by_participants(df, 'white', word_counter, MIN_COUNT_BEFORE_REP))
    if SAVE_MODELS:
        wm.save(white_model_file)

    print("Training black")
    bm.train(repeat_by_participants(df, 'black', word_counter, MIN_COUNT_BEFORE_REP))
    if SAVE_MODELS:
        bm.save(black_model_file)

    print("Training neutral")
    neutral.train(df['tokenized'])
    if SAVE_MODELS:
        neutral.save(neutral_model_file)

Training white
repetitions histogram: defaultdict(<class 'int'>, {20: 6108, 1: 193, 10: 2286, 0: 782})
remaining abstracts after removing rare words and then empty abstracts: 9369, avg length: 326.2285195858683
training model
Training black
repetitions histogram: defaultdict(<class 'int'>, {20: 1890, 0: 1385, 10: 3919, 1: 2175})
remaining abstracts after removing rare words and then empty abstracts: 9369, avg length: 326.2285195858683
training model
Training neutral
training model


In [196]:
# Training FastText
if READ_MODELS:
    wm_ft.load(white_model_file_ft)
    bm_ft.load(black_model_file_ft)
    neutral_ft.load(neutral_model_file_ft)
else:
    print("Training white")
    wm_ft.train(repeat_by_participants(df, 'white', word_counter, MIN_COUNT_BEFORE_REP))
    if SAVE_MODELS:
        wm_ft.save(white_model_file_ft)

    print("Training black")
    bm_ft.train(repeat_by_participants(df, 'black', word_counter, MIN_COUNT_BEFORE_REP))
    if SAVE_MODELS:
        bm_ft.save(black_model_file_ft)

    print("Training neutral")
    neutral_ft.train(df['tokenized'])
    if SAVE_MODELS:
        neutral_ft.save(neutral_model_file_ft)

Training white
repetitions histogram: defaultdict(<class 'int'>, {20: 6108, 1: 193, 10: 2286, 0: 782})
remaining abstracts after removing rare words and then empty abstracts: 9369, avg length: 326.2285195858683
training model
Training black
repetitions histogram: defaultdict(<class 'int'>, {20: 1890, 0: 1385, 10: 3919, 1: 2175})
remaining abstracts after removing rare words and then empty abstracts: 9369, avg length: 326.2285195858683
training model
Training neutral
training model


In [184]:
vocab = wm.get_vocab_list()
type(vocab)
print(vocab[:10])
matrix = wm.get_word_vectors_as_matrix_rows(vocab)
type(matrix)
print(matrix[:1])
# for i in range(len(vocab)):
#     vector_as_str = ",".join([str(x) for x in matrix[i]])


['<NUMBER>', 'and', 'the', 'of', 'be', 'in', 'with', 'to', 'a', 'patient']
[[-1.7353948   0.8484482   4.6544433   0.75861704 -2.6332548   3.4562976
   5.602589   -0.7903625  -0.7725534   5.0008044   0.5056282  -0.1787354
   6.1274242  -3.286178    0.94354737 -3.0432115   3.9792848   1.5576125
  -2.0053916  -3.9694502   0.45551464 -1.0973371   1.9482943   2.6629245
  -0.37642255  3.0030043   3.451346   -0.1183093   1.9901967  -5.1725383
   0.45941284  0.6669147   2.7764049   3.0361755  -0.7754488   1.81441
   5.752063    3.777228   -6.426763   -5.0876837 ]]


In [185]:
wm.export_embeddings('embedding_models/white{}_emb.tsv'.format(iteration))
bm.export_embeddings('embedding_models/black{}_emb.tsv'.format(iteration))
neutral.export_embeddings('embedding_models/neutral{}_emb.tsv'.format(iteration))

In [197]:
wm_ft.export_embeddings('embedding_models/white{}_emb_ft.tsv'.format(iteration))
bm_ft.export_embeddings('embedding_models/black{}_emb_ft.tsv'.format(iteration))
neutral_ft.export_embeddings('embedding_models/neutral{}_emb_ft.tsv'.format(iteration))

In [166]:
# wm.get_vocab_list()
test_word = 'cardiovascular'
print(wm.most_similar(test_word))
print(bm.most_similar(test_word))
print(neutral.most_similar(test_word))

[('cv', 0.8469335436820984), ('cvd', 0.7634232044219971), ('cerebrovascular', 0.7224986553192139), ('ischemic', 0.7102844715118408), ('exscel', 0.7100193500518799)]
[('cv', 0.859120786190033), ('cerebrovascular', 0.7296085953712463), ('cardiorenal', 0.7134647965431213), ('kidney', 0.713179886341095), ('renal', 0.688029944896698)]
[('cv', 0.8210580348968506), ('cvd', 0.7512128949165344), ('cardiorenal', 0.6985706686973572), ('cerebrovascular', 0.6953392624855042), ('thromboembolic', 0.691645622253418)]


In [193]:
# wm.w2v_model.wv.get_vector("not_rock")
wm.w2v_model.wv["rock"]

array([ 0.31689957, -1.5706038 , -0.04648763,  1.2012587 ,  1.7411898 ,
        0.42982486,  0.9022166 , -1.2952621 ,  0.45089534, -1.378571  ,
       -0.27359325,  1.1756372 ,  1.629065  ,  1.7001607 ,  1.1596785 ,
       -1.3816754 ,  0.4724258 ,  0.02134321, -2.6922083 ,  0.7436854 ,
       -0.6102765 ,  2.032709  ,  0.85509086, -0.7137248 , -0.5361561 ,
        0.41292498, -0.3455988 , -0.66979975,  2.1843855 ,  1.0821238 ,
        0.894895  , -0.20694093,  0.29442215,  0.73406005,  6.7623315 ,
       -0.07779735,  0.1509057 , -0.5197101 ,  0.08014201,  0.5338471 ],
      dtype=float32)

In [199]:
wm_ft.w2v_model.wv["rock"]


array([ 1.9651371 ,  1.0653362 , -9.655469  ,  1.2487043 , -1.795831  ,
       -1.7551631 , -0.78154355,  8.6640625 ,  1.6558295 , -1.7618943 ,
       -5.340287  , -8.764476  , -1.5597317 , -5.378958  , -1.6592234 ,
       -2.8142374 , -3.7427063 ,  2.497033  , -8.534532  ,  4.2789073 ,
       -8.950303  ,  3.4331825 ,  6.518765  , -1.9991143 ,  2.1695185 ,
        9.093766  ,  2.835913  , -2.9638727 ,  1.3270564 , -0.14580275,
        2.9196432 ,  1.343975  , -1.8661782 , -2.0862482 , -1.0064353 ,
       -1.656522  , -0.7469466 , -8.497369  , -3.8384511 , -2.165336  ],
      dtype=float32)

In [201]:
wm_ft.w2v_model.wv["I am testing this no_rock"]

array([ 0.4843087 ,  0.32290173, -1.9458232 ,  0.06127902, -0.08210611,
        0.3762979 , -0.29447314,  0.44749537,  1.2849654 , -1.0742685 ,
       -1.1516773 ,  0.48834398, -1.0212396 ,  0.35794872,  1.1873664 ,
       -1.4268283 , -0.16637616,  0.504515  , -1.2366754 , -0.04347903,
       -0.4892661 , -1.1753019 ,  0.9284686 , -0.52430177,  0.59812415,
        2.3847244 , -0.65613836, -0.3214568 ,  0.71342236,  0.16916566,
        0.6364456 ,  1.1190718 ,  0.05863871, -0.40994495,  1.9094901 ,
        0.13981813, -0.33536372,  0.0369867 , -0.57299656, -0.41981992],
      dtype=float32)

# https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

# race sensitive model

In [202]:
READ_MODELS = False
SAVE_MODELS = True

ONLY_CONCEPTS = False
MODE = "plain_text"  # 'plain_text', 'cui2vec_style' or 'only_concepts'
MIN_COUNT = 0
MIN_COUNT_BEFORE_REP = 0 # 40
VECTOR_SIZE = 100  # 50, 70, 100, 300
TRAINING_ITERATIONS = 20  # 35
WINDOW = 20 # 20
TOPN = 10

In [203]:
race_sensitive_model_file_template = "embedding_models/race-sensitive{}.model"
neutral_model_file_template = "embedding_models/neutral{}.model"

In [204]:
iterations = ['_plain_text_FT_vec100_win20_ti20']
neutral_ft, race_sensitive_ft = [], []
for iteration in iterations:
    race_sensitive_model_file_ft = race_sensitive_model_file_template.format(iteration)
    neutral_model_file_ft = neutral_model_file_template.format(iteration)

    race_sensitive_ft = FastTextEmbedModelWrapper(min_count=MIN_COUNT,
                           vector_size=VECTOR_SIZE, iterations=TRAINING_ITERATIONS,
                           window=WINDOW)
    neutral_ft = FastTextEmbedModelWrapper(min_count=MIN_COUNT,
                                vector_size=VECTOR_SIZE, iterations=TRAINING_ITERATIONS,
                                window=WINDOW)

In [205]:
def participants_to_repetitions_ratio(participants):
    if participants == 0:
        repetitions = 0
    elif participants < 0.1:
        repetitions = 1
    elif participants < 0.2:
        repetitions = 2
    elif participants < 0.4:
        repetitions = 5
    elif participants < 0.8:
        repetitions = 10
    else:
        repetitions = 20
    return repetitions


def participants_to_repetitions_ratio(participants):
    if participants == 0:
        repetitions = 0
    elif participants <= 0.1:
        repetitions = 1
    elif participants <= 0.2:
        repetitions = 4
    elif participants <= 0.4:
        repetitions = 8
    elif participants <= 0.8:
        repetitions = 10
    else:
        repetitions = 20
    return repetitions

def repeat_by_participants_ratio(df, weight_col, word_counter, threshold,
                           policy=participants_to_repetitions_ratio):
    tokenized = []
    reps = defaultdict(int)
    total_len = 0
    for i, r in df.iterrows():
        word_list = r['tokenized']
        remaining = [word for word in word_list if word_counter[word] >= threshold]
        if len(remaining) < 2:
            reps['empty'] += 1
            continue
        total_len += len(remaining)
        repetitions = policy(r[weight_col]/max(r['white'], 0.01))
        #DEBUG
        reps[repetitions] += 1
        for _ in range(repetitions):
            tokenized.append(remaining)
    # DEBUG
    print("repetitions histogram: {}".format(reps))
    remaining_abstracts = len(df)-reps['empty']
    print("remaining abstracts after removing rare words and then empty abstracts: {}, avg length: {}".format(
          remaining_abstracts,
          total_len/remaining_abstracts))
    random.shuffle(tokenized)
    return tokenized

In [206]:
# Training FastText
if READ_MODELS:
    race_sensitive_ft.load(race_sensitive_model_file_ft)
    neutral_ft.load(neutral_model_file_ft)
else:
    print("Training race-sensitive")
    race_sensitive_ft.train(repeat_by_participants_ratio(df, 'black', word_counter, MIN_COUNT_BEFORE_REP))
    if SAVE_MODELS:
        race_sensitive_ft.save(race_sensitive_model_file_ft)

    print("Training neutral")
    neutral_ft.train(df['tokenized'])
    if SAVE_MODELS:
        neutral_ft.save(neutral_model_file_ft)

Training race-sensitive
repetitions histogram: defaultdict(<class 'int'>, {5: 1139, 0: 1385, 2: 1176, 1: 4013, 20: 1032, 10: 624})
remaining abstracts after removing rare words and then empty abstracts: 9369, avg length: 326.2285195858683
training model
Training neutral
training model


## Extra

In [ ]:


final_sents_list = []
final_pmid_list = {}
test_count = 0
for index, afile in enumerate(tqdm(files_path_list)):
#     print(afile)
    with open(afile, 'r') as f:
        data = json.load(f)
        # print(data)
        data = data[0]
        if 'pmid' in data:
            pmid = data['pmid']
            if pmid in pmid_repeat_dict:
                repetition = max(int(pmid_repeat_dict[pmid]),1)
#                 print(f'repeart {pmid} abstract for: {repetition}')
                if 'sentence_list' in data:
                    cui_sents = [x['sentence'] for x in data['sentence_list']]
                    final_pmid_list[pmid] = cui_sents
                    cui_sents.append('\n')
                    final_sents_list.extend(cui_sents)

print(f'test_count: {test_count}')
key_list = list(final_pmid_list.keys())
mysize = len(key_list)
print(f'mysize: {mysize}')
print(f'final_sents_list length: {len(final_sents_list)}')

In [ ]:
# ncit_pmid_dir = 'data/final_preprocessed_corpus_july_12.csv'

# ncit_pmid_df = pd.read_csv(ncit_pmid_dir)
# print(ncit_pmid_df.head())

In [ ]:
# ncit_pmid_df_simple = ncit_pmid_df[['pmid', 'black' ,   'white','nctid']].copy()
# ncit_pmid_df_simple.head()

In [36]:
# ncit_pmid_df_simple.to_csv('data/ncit_pmid_df_simple.tsv', sep='\t', index=False)

In [42]:
# def get_repetitions(black, white):
#     ratio = int(black)/max(int(white), 1)
#     if ratio == 0:
#         return 1
#     elif ratio < 0.10:
#         return 2
#     elif ratio < 0.25:
#         return 5
#     elif ratio < 0.50:
#         return 10
#     else:
#         return 20
    
# def get_repetitions(black, white):
#     ratio = int(black)/max(int(white), 1)
#     if ratio == 0:
#         return 1
#     elif ratio < 0.10:
#         return 2
#     elif ratio < 0.25:
#         return 4
#     else:
#         return 8

# ncit_pmid_df_simple['repetition'] = ncit_pmid_df_simple.apply(lambda x: get_repetitions(x['black'],x['white']),axis=1)

In [63]:
print(len(pmid_repeat_dict))
cc = 0
for k, v in pmid_repeat_dict.items():
    print(k, v)
    cc+=1
    if cc>10:
        break

8615
23349433 2
22395889 2
22159015 2
20212196 2
17420364 2
34518266 1
33952397 1
32905623 2
32732305 1
32223381 1
32138746 1


In [64]:
import pickle
with open('data/pmid_repeat_dict.pickle', 'wb') as f:
    pickle.dump(pmid_repeat_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [66]:
# rootdir = 'data/raw_data_1960-2022_cui_repleced_v2'
rootdir = 'data/pmed_abstracts_cui_repleced_v2'

files_path_list = []
for root, dirs, files in os.walk(rootdir):
    for name in files:
        # print(name)
        if name.endswith((".json")):
            # print(name)
            full_path = os.path.join(root, name)
            files_path_list.append(full_path)

print(len(files_path_list))

import json
from tqdm.auto import tqdm
import random


final_sents_list = []
final_pmid_list = {}
test_count = 0
for index, afile in enumerate(tqdm(files_path_list)):
#     print(afile)
    with open(afile, 'r') as f:
        data = json.load(f)
        # print(data)
        data = data[0]
        if 'pmid' in data:
            pmid = data['pmid']
            if pmid in pmid_repeat_dict:
                repetition = max(int(pmid_repeat_dict[pmid]),1)
#                 print(f'repeart {pmid} abstract for: {repetition}')
                if 'sentence_list' in data:
                    cui_sents = [x['sentence'] for x in data['sentence_list']]
                    final_pmid_list[pmid] = cui_sents
                    cui_sents.append('\n')
                    final_sents_list.extend(cui_sents)

print(f'test_count: {test_count}')
key_list = list(final_pmid_list.keys())
mysize = len(key_list)
print(f'mysize: {mysize}')
print(f'final_sents_list length: {len(final_sents_list)}')


12795


100%|██████████| 12795/12795 [00:04<00:00, 2926.15it/s]

test_count: 0
mysize: 8615
final_sents_list length: 60198


In [68]:
out_file_name = 'data/raw_data_v3_without_trial_repetitaion_'+str(len(final_sents_list))+'.txt'
with open(out_file_name, 'w') as f:
    for line in final_sents_list:
        f.write(f"{line}\n")

In [69]:
final_sents_list = []
final_pmid_list = {}
test_count = 0
for index, afile in enumerate(tqdm(files_path_list)):
#     print(afile)
    with open(afile, 'r') as f:
        data = json.load(f)
        # print(data)
        data = data[0]
        if 'pmid' in data:
            pmid = data['pmid']
            if pmid in pmid_repeat_dict:
                repetition = max(int(pmid_repeat_dict[pmid]),1)
#                 print(f'repeart {pmid} abstract for: {repetition}')
                if 'sentence_list' in data:
                    cui_sents = [x['sentence'] for x in data['sentence_list']]
                    final_pmid_list[pmid] = cui_sents
                    cui_sents.append('\n')
                    for rep in range(repetition):
                        final_sents_list.extend(cui_sents)

print(f'test_count: {test_count}')
key_list = list(final_pmid_list.keys())
mysize = len(key_list)
print(f'mysize: {mysize}')
print(f'final_sents_list length: {len(final_sents_list)}')


100%|██████████| 12795/12795 [00:04<00:00, 2973.09it/s]

test_count: 0
mysize: 8615
final_sents_list length: 155287


In [70]:
out_file_name = 'data/raw_data_v3_with_trial_repetitaion_'+str(len(final_sents_list))+'.txt'
with open(out_file_name, 'w') as f:
    for line in final_sents_list:
        f.write(f"{line}\n")

In [33]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

count = 0
chunk_size = 94681
for sublist in chunks(final_sents_list, chunk_size):
    count += chunk_size
    final_nonext_list =[]
    for i in tqdm(sublist):
        rand1=0
        rand2=0
        while rand1==rand2:
            rand1 = random.choice(range(mysize))
            rand2 = random.choice(range(mysize))
        sent1 = random.choice(final_pmid_list[key_list[rand1]])
        sent2 = random.choice(final_pmid_list[key_list[rand2]])
        z = '[CLS] ' + str(sent1) + ' [SEP] ' + str(sent2)
        final_nonext_list.append(z)

  
    assert len(sublist) == len(final_nonext_list)
    out_list = [sublist, final_nonext_list]
    out_file_name = 'data/raw_data_v3_with_trial_repetitaion_'+str(count)+'.pkl'
    with open(out_file_name, 'wb') as handle:
        pickle.dump(out_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(f'writing file {out_file_name} done')
        handle.close()
print(f'generate nonext list done.')
        



  0%|          | 0/94681 [00:00<?, ?it/s]

writing file /media/data/nour/data/cui_data_v3_with_trial_repetitaion_94681.pkl done
generate nonext list done.


In [34]:
file_path = '/media/data/nour/data/cui_data_v3_with_trial_repetitaion_94681.pkl'
with open(file_path, 'rb') as handle:
    b = pickle.load(handle)

print(len(b))

2


In [46]:
print(b[0][0])
print(b[1][0])
print('###'*20)
print(b[0][1])
print(b[1][1])
print('###'*20)
print(b[0][2])
print(b[1][2])
print('###'*20)
print(b[0][3])
print(b[1][3])
print('###'*20)
print(b[0][4])
print(b[1][4])

[CLS] TITLE : Predictors of Behavioral Changes After Adenotonsillectomy in Pediatric Obstructive  C0037315 : A Secondary Analysis of a Randomized Clinical Trial. [SEP] Importance : Adenotonsillectomy (AT) is associated with improved behavior in children with obstructive  C0037315  (OSA). However, it is unknown whether polysomnographic parameters are superior to the parent-reported severity of sleep- C0012634  breathing ( C4551689 ) in predicting behavioral changes after AT.
[CLS] INTERVENTION : Randomized allocation of liposomal bupivacaine 4 mg/kg or bupivacaine hydrochloride (HCl) 2 mg/kg via local infiltration at the end of spine surgery (age group 1); liposomal bupivacaine 4 mg/kg via local infiltration at the end of spine or cardiac surgery (age group 2). [SEP] Copyright © 2019. Published by Elsevier Inc.
############################################################
[CLS] TITLE : Predictors of Behavioral Changes After Adenotonsillectomy in Pediatric Obstructive  C0037315 : A Second

## no repetition - baseline


In [52]:
# rootdir = 'data/raw_data_1960-2022_cui_repleced_v3'
rootdir = 'data/pmed_abstracts_cui_repleced_v3'

files_path_list = []
for root, dirs, files in os.walk(rootdir):
    for name in files:
        # print(name)
        if name.endswith((".json")):
            # print(name)
            full_path = os.path.join(root, name)
            files_path_list.append(full_path)

print(len(files_path_list))

import json
from tqdm.auto import tqdm
import random


final_sents_list = []
final_pmid_list = {}
test_count = 0
for index, afile in enumerate(tqdm(files_path_list)):
#     print(afile)
    with open(afile, 'r') as f:
        data = json.load(f)
        # print(data)
        data = data[0]
        if 'pmid' in data:
            pmid = data['pmid']
            if pmid in pmid_repeat_dict:
                repetition = 1   # max(int(pmid_repeat_dict[pmid]),1)
                print(f'repeart {pmid} abstract for: {repetition}')
                if 'sentence_list' in data:
                    cui_sents = [x['sentence_updated'] for x in data['sentence_list']]
                    final_pmid_list[pmid] = cui_sents
                    # print(cui_sents)
                    for x, y in zip(cui_sents, cui_sents[1:]):
                        z = '[CLS] ' + str(x) + ' [SEP] ' + str(y)
                        # print(z)
                        zlist = [z] * repetition
                        test_count += len(zlist)
                        final_sents_list.extend(zlist)

print(f'test_count: {test_count}')
key_list = list(final_pmid_list.keys())
mysize = len(key_list)
print(f'mysize: {mysize}')
print(f'final_sents_list length: {len(final_sents_list)}')


12795


  0%|          | 0/12795 [00:00<?, ?it/s]

repeart 32880655 abstract for: 1
repeart 33568383 abstract for: 1
repeart 26565927 abstract for: 1
repeart 30648126 abstract for: 1
repeart 32335895 abstract for: 1
repeart 32453369 abstract for: 1
repeart 26145274 abstract for: 1
repeart 27178765 abstract for: 1
repeart 29096777 abstract for: 1
repeart 34957550 abstract for: 1
repeart 26564598 abstract for: 1
repeart 27210427 abstract for: 1
repeart 25151488 abstract for: 1
repeart 30488542 abstract for: 1
repeart 24821196 abstract for: 1
repeart 26416154 abstract for: 1
repeart 26160879 abstract for: 1
repeart 28973516 abstract for: 1
repeart 27548026 abstract for: 1
repeart 34571336 abstract for: 1
repeart 23578015 abstract for: 1
repeart 31166697 abstract for: 1
repeart 26500107 abstract for: 1
repeart 30181363 abstract for: 1
repeart 30850240 abstract for: 1
repeart 25677353 abstract for: 1
repeart 26511012 abstract for: 1
repeart 30414528 abstract for: 1
repeart 30661255 abstract for: 1
repeart 33840781 abstract for: 1
repeart 28

In [53]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

count = 0
chunk_size = 42776
for sublist in chunks(final_sents_list, chunk_size):
    count += chunk_size
    final_nonext_list =[]
    for i in tqdm(sublist):
        rand1=0
        rand2=0
        while rand1==rand2:
            rand1 = random.choice(range(mysize))
            rand2 = random.choice(range(mysize))
        sent1 = random.choice(final_pmid_list[key_list[rand1]])
        sent2 = random.choice(final_pmid_list[key_list[rand2]])
        z = '[CLS] ' + str(sent1) + ' [SEP] ' + str(sent2)
        final_nonext_list.append(z)

  
    assert len(sublist) == len(final_nonext_list)
    out_list = [sublist, final_nonext_list]
    out_file_name = '/media/data/nour/data/cui_data_v3_with_trial_baseline_no_repetitaion_'+str(count)+'.pkl'
    with open(out_file_name, 'wb') as handle:
        pickle.dump(out_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(f'writing file {out_file_name} done')
        handle.close()
print(f'generate nonext list done.')

  0%|          | 0/42776 [00:00<?, ?it/s]

writing file /media/data/nour/data/cui_data_v3_with_trial_baseline_no_repetitaion_42776.pkl done
generate nonext list done.


In [54]:
file_path = '/media/data/nour/data/cui_data_v3_with_trial_baseline_no_repetitaion_42776.pkl'
with open(file_path, 'rb') as handle:
    b = pickle.load(handle)

print(len(b))

2


In [55]:
print(b[0][0])
print(b[1][0])
print('###'*20)
print(b[0][1])
print(b[1][1])
print('###'*20)
print(b[0][2])
print(b[1][2])
print('###'*20)
print(b[0][3])
print(b[1][3])
print('###'*20)
print(b[0][4])
print(b[1][4])

[CLS] TITLE : Predictors of Behavioral Changes After Adenotonsillectomy in Pediatric Obstructive  C0037315 : A Secondary Analysis of a Randomized Clinical Trial. [SEP] Importance : Adenotonsillectomy (AT) is associated with improved behavior in children with obstructive  C0037315  (OSA). However, it is unknown whether polysomnographic parameters are superior to the parent-reported severity of sleep- C0012634  breathing ( C4551689 ) in predicting behavioral changes after AT.
[CLS] RESULTS : In all, 699 women living with HIV were recruited during pregnancy (8-24 weeks), and assessments were completed at baseline, at 32 weeks pregnant (61.7%), and at 6 weeks (47.6%), 6 months (50.6%) and 12 months (59.5%) postnatally. Infants were tested for HIV at 6 weeks and 12 months, 73.5% living infants were tested at 6 weeks and 56.7% at 12 months. There were no significant differences between SC and EI on infant HIV status at 6 weeks and at 12 months, and no differences in maternal adherence at 32 

## Extra 

In [51]:
b= []
c = 'abc'
b.extend(c)
print(b)

['a', 'b', 'c']


In [49]:
a = [1,2,3,4,5]
b = a* 5
print(b)

[1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5]
